# Sarcasm Detection using Tensorflow - Bidirectional LSTMs and Convolutional 1D

Past studies in Sarcasm Detection mostly make use of Twitter datasets collected using hashtag based supervision but such datasets are noisy in terms of labels and language. Furthermore, many tweets are replies to other tweets and detecting sarcasm in these requires the availability of contextual tweets.

To overcome the limitations related to noise in Twitter datasets, this News Headlines dataset for Sarcasm Detection is collected from two news website. TheOnion aims at producing sarcastic versions of current events and we collected all the headlines from News in Brief and News in Photos categories (which are sarcastic). We collect real (and non-sarcastic) news headlines from HuffPost.

This new dataset has following advantages over the existing Twitter datasets:

Since news headlines are written by professionals in a formal manner, there are no spelling mistakes and informal usage. This reduces the sparsity and also increases the chance of finding pre-trained embeddings.

Furthermore, since the sole purpose of TheOnion is to publish sarcastic news, we get high-quality labels with much less noise as compared to Twitter datasets.

Unlike tweets which are replies to other tweets, the news headlines we obtained are self-contained. This would help us in teasing apart the real sarcastic elements.

**Content**
Each record consists of three attributes:

is_sarcastic: 1 if the record is sarcastic otherwise 0

headline: the headline of the news article

article_link: link to the original news article. Useful in collecting supplementary data

***
The concept is to perform tokenizing the words to get numeric values from them and then using Embeddings to group words of similar meaning,
depending on how they are labelled resulting in a good but rough sentiment analysis
***

In [1]:
# Library imports

import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Load the json data

def parse_data(file):
    for l in open(file,'r'):
        yield json.loads(l)

datastore = list(parse_data('resources/Sarcasm_Headlines_Dataset.json'))

# define the attribues from the json data
sentences = []
labels = []
urls = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [3]:
# Define some global constants
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<oov>"
training_size = 20000

In [6]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [7]:
# Build the tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [11]:
# Build the model and compile
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_2 (Dense)              (None, 24)                1560      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 30,129
Trainable params: 30,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# fit on the training data
num_epochs = 50
training_padded = np.array(training_padded)
testing_padded = np.array(testing_padded)
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)
history = model.fit(training_padded, training_labels, epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels), verbose=1)

2021-12-13 18:46:37.373557: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/50
625/625 [==============================] - 21s 29ms/step - loss: 0.5521 - accuracy: 0.6824 - val_loss: 0.3861 - val_accuracy: 0.8223
Epoch 2/50
625/625 [==============================] - 19s 31ms/step - loss: 0.3569 - accuracy: 0.8337 - val_loss: 0.3776 - val_accuracy: 0.8241
Epoch 3/50
625/625 [==============================] - 18s 28ms/step - loss: 0.3357 - accuracy: 0.8464 - val_loss: 0.3712 - val_accuracy: 0.8299
Epoch 4/50
625/625 [==============================] - 16s 25ms/step - loss: 0.3156 - accuracy: 0.8624 - val_loss: 0.3710 - val_accuracy: 0.8301
Epoch 5/50
625/625 [==============================] - 15s 24ms/step - loss: 0.3056 - accuracy: 0.8642 - val_loss: 0.3763 - val_accuracy: 0.8326
Epoch 6/50
625/625 [==============================] - 16s 25ms/step - loss: 0.2965 - accuracy: 0.8670 - val_loss: 0.3773 - val_accuracy: 0.8346
Epoch 7/50
625/625 [==============================] - 15s 25ms/step - loss: 0.2806 - accuracy: 0.8783 - val_loss: 0.3770 - val_accuracy: